In [4]:
from IPython.display import HTML
HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [5]:
'''###################### Requirements #####################'''
    
import re
import os
import PIL
import csv
import time
global shell
import string
import openpyxl
import datetime
import win32gui
import warnings
import itertools
import numpy as np
import pandas as pd
from tkinter import *
from os import listdir
import win32com.client
import pyautogui as pag
import ipywidgets as widgets
from datetime import datetime
from openpyxl import Workbook
from ipywidgets import Layout
from tkinter import messagebox
from os.path import isfile, join
from openpyxl.formatting import Rule
from stat import S_ISREG, ST_CTIME, ST_MODE
from openpyxl.worksheet.cell_range import CellRange
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import ColorScaleRule, CellIsRule, FormulaRule
from openpyxl.styles import Alignment, Border, Side, Font, Color, PatternFill
warnings.simplefilter(action='ignore', category=FutureWarning)
def printred(skk): print("\033[91m {}\033[00m" .format(skk))
    

'''###################### Little snippet to manage the windows UI #####################'''


class WindowMgr:

    def __init__ (self):
        self._handle = None

    def find_window(self, class_name, window_name=None):
        self._handle = win32gui.FindWindow(class_name, window_name)

    def _window_enum_callback(self, hwnd, wildcard):
        if re.match(wildcard, str(win32gui.GetWindowText(hwnd))) is not None:
            self._handle = hwnd

    def find_window_wildcard(self, wildcard):
        self._handle = None
        win32gui.EnumWindows(self._window_enum_callback, wildcard)

    def set_foreground_k(self):   
        shell = win32com.client.Dispatch("WScript.Shell")
        shell.SendKeys(' ')
        win32gui.SetForegroundWindow(self._handle)
    
    def set_foreground(self):   
        win32gui.SetForegroundWindow(self._handle)


'''###################### Preliminary functions #####################'''


class first_step:
    
    def create_folders():

            global path0, path_report1, path_report2, path_results, path_fails

            path0 = "C:\\Users\\MaiocchiMarco\\Desktop\\PORT_RT\\" + datetime.today().strftime("%m%d%y") + "\\"
            path_report1 = path0 + "REPORT_1"
            path_report2 = path0 + "REPORT_2"
            path_results = path0 + "RESULTS"
            path_fails = path0 + "FAILURES"

            if not os.path.exists(path0):
                os.makedirs(path0)
            if not os.path.exists(path_report1):
                os.mkdir(path_report1)
            if not os.path.exists(path_report2):
                os.mkdir(path_report2)
            if not os.path.exists(path_results):
                os.mkdir(path_results)
            if not os.path.exists(path_fails):
                os.mkdir(path_fails)    
            
    def read_inputs_from_ctrl_file():
        global df_custom
        xls = pd.ExcelFile("C:\\Users\\MaiocchiMarco\\Desktop\\Control_file_PORT_RT.xlsx")
        if tests_type.value == 'CUSTOM':
            df_custom = pd.read_excel(xls, 'CUSTOM')
        elif tests_type.value == 'NXQC':
            df_custom = pd.read_excel(xls, 'NXQC')
        elif tests_type.value == 'RISK':
            df_custom = pd.read_excel(xls, 'RISK')
        elif tests_type.value == 'HIPPO':
            df_custom = pd.read_excel(xls, 'HIPPO')
        
    def setup_excel_template():
        global workbook, sheet, recap_sheet
        workbook = Workbook()
        sheet = workbook.active
        recap_sheet = workbook['Sheet']
        recap_sheet.title = 'Results'
        recap_sheet.sheet_view.showGridLines = False
        recap_sheet['B2'].value = 'TESTS RESULTS'
        recap_sheet['B4'].value = 'Test Name'
        recap_sheet['C4'].value = '% Errors'
        recap_sheet.column_dimensions['B'].width = 40
        recap_sheet.column_dimensions['C'].width = 23
        for row in range(4,1000):
            recap_sheet.cell(row, 3).alignment = Alignment(horizontal='center', wrap_text=False)


'''###################### Functions to automate PORT report generation with inputs #####################'''


class bbg_rt:
    
    def bbg_window_open():
        global win
        win = '1-BLOOMBERG'
        w.find_window_wildcard(".*" + win +".*")
        w.set_foreground_k()
        time.sleep(1)
        
    def bbg_window_select():
        w.find_window_wildcard(".*" + win +".*")
        w.set_foreground()
        time.sleep(1)
        
    def iter_tab(n):
        for _ in itertools.repeat(None, n):
            time.sleep(0.2)
            pag.press('tab')
            
    def selectptf():
        bbg_rt.bbg_window_select()
        pag.write(ptf)
        time.sleep(0.5)
        pag.press('f12')
        time.sleep(0.5)
        pag.press('enter')
    
    def openviewprod():
        bbg_rt.bbg_window_select()
        time.sleep(5)
        pag.write("PORT " + tab + " V " + view + " /QA")
        time.sleep(0.5)
        pag.press('enter')
        pag.press('enter')
        time.sleep(15)
         
    def openviewqa():  
        bbg_rt.bbg_window_select()    
        time.sleep(5)
        pag.write("RRRR PORT " + m2 + " " + tab + " V " + view + " /QA")
        time.sleep(0.5)
        pag.press('enter')
        pag.press('enter')
        time.sleep(15)
           
    def chooseTab():
        bbg_rt.bbg_window_select()
        
        if tab == 'HP':
            if subt == 'MainView':
                pag.write('30')
            elif subt == 'TotalReturn':
                pag.write('31')
            elif subt == 'PeriodAnalysis':
                pag.write('32')
            elif subt == 'SeasonalAnalysis':
                pag.write('33')
            elif subt == 'StatisticalSummary':  
                pag.write('34')
            time.sleep(2)
            pag.press('enter')
            time.sleep(5)
        
        else:
            if subt == 'MainView':
                pag.write('30')
            elif subt == 'Summary':
                pag.write('31')
            elif subt == 'Factors':
                pag.write('32')
            elif subt == 'RiskBets':
                pag.write('33')
            elif subt == 'Trends':  
                pag.write('34')
            elif subt == 'Exposures':
                pag.write('35')
            time.sleep(2)
            pag.press('enter')
            time.sleep(5)
        
    def change_bmk(n):
        
        if bmk == "Default":
            pass
        else:
            bbg_rt.bbg_window_select()
            bbg_rt.iter_tab(n)
            time.sleep(1)
            pag.write(bmk)
            time.sleep(1)
            pag.press('enter')
            pag.press('enter')
            time.sleep(2)
            
    def change_bkdn(n):
        
        if bkdn == "Default":
            pass
        else:
            bbg_rt.bbg_window_select()
            bbg_rt.iter_tab(n)
            time.sleep(1)
            pag.write(bkdn)
            time.sleep(1)
            pag.press('enter')
            pag.press('enter')
            time.sleep(2)
            
    def change_ccy(n):
   
        if ccy == "Default":
            pass
        else:
            bbg_rt.bbg_window_select()
            bbg_rt.iter_tab(n)
            time.sleep(1)
            pag.write(ccy)
            time.sleep(1)
            pag.press('enter')
            pag.press('enter')
            time.sleep(2)
    
    def change_day(n):
        
        if days_list.value == 'PREV_CLOSE':
            pass
        else:

            mm = day.split('/')[0]
            dd = day.split('/')[1]
            yy = day.split('/')[2]

            bbg_rt.bbg_window_select()
            bbg_rt.iter_tab(n)
            pag.write(mm)
            time.sleep(1)
            pag.write(dd)
            time.sleep(1)
            pag.write(yy)
            time.sleep(1)
            pag.press('enter')
            pag.press('enter')
            time.sleep(2)
            
    def change_model(n):
        
        if model == 'Default':
            pass
        else:
            if tab in ['TE','VR','SA']:
                bbg_rt.bbg_window_select()
                bbg_rt.iter_tab(n)
                time.sleep(1)
                pag.write(model)
                time.sleep(1)
                pag.press('enter')
                pag.press('enter')
                time.sleep(2)
            else:
                pass
            
    def change_unit(n):
        
        if unit == 'Default':
            pass
        else:
            if tab in ['TE','VR']:
                bbg_rt.bbg_window_select()
                bbg_rt.iter_tab(n)
                time.sleep(1)
                pag.write(unit)
                time.sleep(1)
                pag.press('enter')
                pag.press('enter')
                time.sleep(2)
            else:
                pass
            
    def change_clvl(n):
        
        if clvl == 'Default':
            pass
        else:
            if tab in ['TE','VR']:
                bbg_rt.bbg_window_select()
                bbg_rt.iter_tab(n)
                time.sleep(1)
                pag.write(clvl)
                time.sleep(1)
                pag.press('enter')
                pag.press('enter')
                time.sleep(2)
            else:
                pass
            
    def change_hz(n):
        
        if hz == 'Default':
            pass
        else:
            if tab in ['TE','VR']:
                bbg_rt.bbg_window_select()
                bbg_rt.iter_tab(n)
                time.sleep(1)
                pag.write(hz)
                time.sleep(1)
                pag.press('enter')
                pag.press('enter')
                time.sleep(2)
            else:
                pass
            
    def change_attrb(n):

        if model == 'Default':
            pass
        else:
            if tab == 'PA':
                bbg_rt.bbg_window_select()
                bbg_rt.iter_tab(n)
                time.sleep(1)
                pag.write(model)
                time.sleep(1)
                pag.press('enter')
                pag.press('enter')
                time.sleep(2)
            else:
                pass
    
    def change_scen(n):
        
        if scen == 'Default':
            pass
        else:
            if tab == 'SA':
                bbg_rt.bbg_window_select()
                bbg_rt.iter_tab(n)
                time.sleep(1)
                pag.write(scen)
                time.sleep(1)
                pag.press('enter')
                pag.press('enter')
                time.sleep(2)
            else:
                pass
              
    def change_riskversion(n):  #only if enabled for MAC3
        
        bbg_rt.bbg_window_select()
        bbg_rt.iter_tab(n)
        time.sleep(1)
        pag.write('MAC3')
        time.sleep(1)
        pag.press('enter')
        pag.press('enter')
        time.sleep(2)
                  
    def export():
        
        bbg_rt.bbg_window_select()
        
        if tmp == 'Current Tab (Unformatted xls)':
        
            time.sleep(5)
            pag.write('12')
            pag.press('enter')
            time.sleep(1)

            pag.write('6')
            pag.press('enter')
            time.sleep(1)

            pag.write('1')
            pag.press('enter')
            time.sleep(1)
            
        else:
            
            time.sleep(10)
            pag.write('12')
            pag.press('enter')
            time.sleep(1)

            pag.write('6')
            pag.press('enter')
            time.sleep(1)

            pag.write('2')
            pag.press('enter')
            time.sleep(1)
            
    def try_grab_report():
        
        x = 0
        while x < 5:
            try:
                x = x + 1
                time.sleep(5)
                w.find_window_wildcard(".*grid.*")
                w.set_foreground()
                break
            except:
                continue
              
    def export_loop_mainview():
        
        while True:
            try:
                x = 0
                while x < 5:
                    x = x + 1
                    try: #try to grab the report 5 times
                        time.sleep(5)
                        w.find_window_wildcard(".*grid.*")
                        w.set_foreground()
                        break
                    except:
                        continue
                #load it to see inside
                file = win32gui.GetWindowText(win32gui.GetForegroundWindow()).split()[0]
                os.chdir("C:\\blp\\data\\")
                wb = openpyxl.load_workbook(file + '.xlsx')
                sheet = wb['Worksheet']
                y = 0
                #if empty export again for maximum 5 times
                while str(str(sheet['B2'].value) + str(sheet['B3'].value) + str(sheet['B4'].value) +
                          str(sheet['B5'].value) + str(sheet['B6'].value) + str(sheet['C2'].value) +
                          str(sheet['C3'].value) + str(sheet['C4'].value) + str(sheet['C5'].value) +
                          str(sheet['C6'].value) + str(sheet['D2'].value) + str(sheet['D3'].value) +
                          str(sheet['D4'].value) + str(sheet['D5'].value) + str(sheet['D6'].value) +
                          str(sheet['E2'].value)) == 'NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone' and y < 10 :
                    y = y + 1
                    bbg_rt.closeGridfile()
                    bbg_rt.export()
                    z = 0
                    while z < 5:
                        z = z + 1
                        try:
                            time.sleep(5)
                            w.find_window_wildcard(".*grid.*")
                            w.set_foreground()
                            break
                        except:
                            continue
                    file = win32gui.GetWindowText(win32gui.GetForegroundWindow()).split()[0]
                    os.chdir("C:\\blp\\data\\")
                    wb = openpyxl.load_workbook(file + '.xlsx')
                    sheet = wb['Worksheet']
                #kicks an error if tried to export not-empty report more than 5 times
                if y == 10:
                    bbg_rt.closeGridfile()
                else:
                    pass
                break
            except:
                break
        
    def export_loop_else():
        
        export_iter = 0
        while export_iter < 5:
            export_iter = export_iter +  1
            x = 0
            while x < 5:
                x = x + 1
                try: 
                    time.sleep(5)
                    w.find_window_wildcard(".*grid.*")
                    w.set_foreground()
                    break
                except:
                    continue
            file_check = win32gui.GetWindowText(win32gui.GetForegroundWindow()).split()[0][0:4]
            if file_check == 'grid':
                file = win32gui.GetWindowText(win32gui.GetForegroundWindow()).split()[0]
                os.chdir("C:\\blp\\data\\")
                wb = openpyxl.load_workbook(file + '.xlsx')
                sheet = wb['Worksheet']
                y = 0
                #if empty export again for maximum 5 times
                while str(str(sheet['B2'].value) + str(sheet['B3'].value) + str(sheet['B4'].value) +
                          str(sheet['B5'].value) + str(sheet['B6'].value) + str(sheet['C2'].value) +
                          str(sheet['C3'].value) + str(sheet['C4'].value) + str(sheet['C5'].value) +
                          str(sheet['C6'].value) + str(sheet['D2'].value) + str(sheet['D3'].value) +
                          str(sheet['D4'].value)) == 'NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone' and y < 10 :
                    y = y + 1
                    bbg_rt.closeGridfile()
                    bbg_rt.export()
                    z = 0
                    while z < 5:
                        z = z + 1
                        try:
                            time.sleep(5)
                            w.find_window_wildcard(".*grid.*")
                            w.set_foreground()
                            break
                        except:
                            continue
                    file = win32gui.GetWindowText(win32gui.GetForegroundWindow()).split()[0]
                    os.chdir("C:\\blp\\data\\")
                    wb = openpyxl.load_workbook(file + '.xlsx')
                    sheet = wb['Worksheet']
                #kicks an error if tried to export not-empty report more than 5 times
                if y == 10:
                    bbg_rt.closeGridfile()
                else:
                    pass
                break
            else:
                bbg_rt.export()
                time.sleep(5)
                continue
            
    def export_loop_formatted():
        
        x = 0
        while x < 30:
            x = x + 1
            try:
                time.sleep(5)
                w.find_window_wildcard(".* Excel.*")
                w.set_foreground()
                break
            except:
                continue
                       
    def export_loop():
        
        if tmp == 'Current Tab (Unformatted xls)':
        
            if subt == 'MainView':
                bbg_rt.export_loop_mainview()
            else:
                bbg_rt.export_loop_else()
                                
        else:
            bbg_rt.export_loop_formatted()
                        
    def dumpXLS1(_id):
        if tmp == 'Current Tab (Unformatted xls)': 
            time.sleep(2)
            file = win32gui.GetWindowText(win32gui.GetForegroundWindow()).split()[0]
            os.chdir("C:\\blp\\data\\")
            wb = openpyxl.load_workbook(file + '.xlsx')
            wb.save(filename = path_report1 + '\\1_' + tab + '_' + subt + '_' + ptf + '_' + str(_id) +'.xlsx')
        else:
            time.sleep(5)
            file = win32gui.GetWindowText(win32gui.GetForegroundWindow())[0:8]
            path = "C:\\blp\\data\\"
            fname = path + file + ".xls"
            excel = win32com.client.gencache.EnsureDispatch('Excel.Application')
            wb = excel.Workbooks.Open(fname)
            wb.SaveAs(path_report1 + '\\1_' + tab + '_' + subt + '_' + ptf + '_' + str(_id) +'.xlsx', FileFormat = 51)
                                          
    def dumpXLS2(_id):
        if tmp == 'Current Tab (Unformatted xls)': 
            time.sleep(5)
            file = win32gui.GetWindowText(win32gui.GetForegroundWindow()).split()[0]
            os.chdir("C:\\blp\\data\\")
            wb = openpyxl.load_workbook(file + '.xlsx')
            wb.save(filename = path_report2 + '\\2_' + tab + '_' + subt + '_' + ptf + '_' + str(_id) +'.xlsx')
        else:
            time.sleep(5)
            file = win32gui.GetWindowText(win32gui.GetForegroundWindow())[0:8]
            path = "C:\\blp\\data\\"
            fname = path + file + ".xls"
            excel = win32com.client.gencache.EnsureDispatch('Excel.Application')
            wb = excel.Workbooks.Open(fname)
            wb.SaveAs(path_report2 + '\\2_' + tab + '_' + subt + '_' + ptf + '_' + str(_id) +'.xlsx', FileFormat = 51)
                                 
    def closeGridfile():
        if tmp == 'Current Tab (Unformatted xls)': 
            w.find_window_wildcard(".*grid.*")
            w.set_foreground()
            time.sleep(1)
            pag.hotkey('ctrl', 'f4')
            time.sleep(3)
        else:
            w.find_window_wildcard(".* Excel.*")
            w.set_foreground()
            time.sleep(1)
            pag.hotkey('ctrl', 'f4')
            time.sleep(3)
    
    def closeExcel():
        try:
            w.find_window_wildcard(".* Excel.*")
            w.set_foreground()
            pag.hotkey('Alt', 'f4')
        except:
            pass
        
        
'''###################### Add a custom comparison or analysis #####################'''


class analyze:
    
    def round_float(s):
        m = re.match("(\d+\.\d+)",s.__str__())
        try:
            r = round(float(m.groups(0)[0]),6)
        except:
            r = s
        return r
    
    def round_df(df):
        for col in range(0,len(df.columns)):
            df[col] = df[col].apply(analyze.round_float)
                 
    def compute_differences(r):
        
        global df1, df2, df_comparison
        if tmp == 'Current Tab (Unformatted xls)':
            df1 = pd.read_excel(path_report1 + '\\1_' + tab + '_' + subt + '_' + ptf + '_' + str(r) + '.xlsx', header = None).fillna(0).iloc[:1000]
            df2 = pd.read_excel(path_report2 + '\\2_' + tab + '_' + subt + '_' + ptf + '_' + str(r) + '.xlsx', header = None).fillna(0).iloc[:1000]
        else:
            df1 = pd.read_excel(path_report1 + '\\1_' + tab + '_' + subt + '_' + ptf + '_' + str(r) + '.xlsx', header = None).fillna(0)[:-1]
            df2 = pd.read_excel(path_report2 + '\\2_' + tab + '_' + subt + '_' + ptf + '_' + str(r) + '.xlsx', header = None).fillna(0)
            df2 = df2.iloc[:,0:len(df2.columns)+(df1.shape[1]-df2.shape[1])] #fix to avoid styling issues in QA
            
        for col in range(0,len(df1.columns)):
            df1[col] = df1[col].apply(analyze.round_float)
        for col in range(0,len(df2.columns)):
            df2[col] = df2[col].apply(analyze.round_float)
            
        df_comparison = (df1 == df2)
        for c in range(0,len(df_comparison.columns)):
            for rw in range(0,len(df_comparison.index)):
                if df_comparison.iloc[rw,c] == True:
                    df_comparison.iloc[rw,c] = df1.iloc[rw,c]
                else:
                    df_comparison.iloc[rw,c] = str(df1.iloc[rw,c]) + " <> " + str(df2.iloc[rw,c])
                    
    def compute_differences_v8(r):
        
        global df1, df2, df_comparison
        if tmp == 'Current Tab (Unformatted xls)':
            df1 = pd.read_excel(path_report1 + '\\1_' + tab + '_' + subt + '_' + ptf + '_' + str(r) + '.xlsx', header = None).fillna(0).iloc[:1000]
            df2 = pd.read_excel(path_report2 + '\\2_' + tab + '_' + subt + '_' + ptf + '_' + str(r) + '.xlsx', header = None).fillna(0).iloc[:1000]
        else:
            df1 = pd.read_excel(path_report1 + '\\1_' + tab + '_' + subt + '_' + ptf + '_' + str(r) + '.xlsx', header = None).fillna(0)[:-1]
            df2 = pd.read_excel(path_report2 + '\\2_' + tab + '_' + subt + '_' + ptf + '_' + str(r) + '.xlsx', header = None).fillna(0)
            df2 = df2.iloc[:,0:len(df2.columns)+(df1.shape[1]-df2.shape[1])] #fix to avoid styling issues in QA
            
        for col in range(0,len(df1.columns)):
            df1[col] = df1[col].apply(analyze.round_float)
        for col in range(0,len(df2.columns)):
            df2[col] = df2[col].apply(analyze.round_float)
            
        df_comparison = df1.copy()
        for c in range(0,len(df_comparison.columns)):
            for rw in range(0,len(df_comparison.index)):
                try: 
                    if abs(df1.iloc[rw,c]/df2.iloc[rw,c]-1) > 0.5:
                        df_comparison.iloc[rw,c] = str(df1.iloc[rw,c]) + " <> " + str(df2.iloc[rw,c])
                    else:
                        pass
                except:
                    try:
                        if df1.iloc[rw,c] == df2.iloc[rw,c]:
                            pass
                        else:
                            df_comparison.iloc[rw,c] = str(df1.iloc[rw,c]) + " <> " + str(df2.iloc[rw,c])
                    except:
                        pass
           
    def dump_results_to_excel(r):
       
        global sheet_new
        sheet_new = workbook.create_sheet(str(r))
        sheet_new.sheet_view.showGridLines = False
        
        rows = dataframe_to_rows(df_comparison)
        for r_idx, row in enumerate(rows, 1):
            for c_idx, value in enumerate(row, 1):
                 sheet_new.cell(row=r_idx, column=c_idx, value=value)
                   
    def add_formatting():

        red_text = Font(color="9C0006")
        red_fill = PatternFill(bgColor="FFC7CE")
        dxf = DifferentialStyle(font=red_text, fill=red_fill)
        rule = Rule(type="containsText", operator="containsText", text="<>", dxf=dxf)
        rule.formula = ['NOT(ISERROR(SEARCH("<>",A1)))']
        sheet_new.conditional_formatting.add('A1:ZZ5000', rule)
        for row in sheet_new['A1:ZZ2']:
            for cell in row:
                cell.value = None
        for row in sheet_new['A1:A10000']:
            for cell in row:
                cell.value = None 
        
    def recap_results(r):
        
        try:
            recap_sheet['B' + str(r+5)].value = str(r) + '_' + check + '_' + ptf + '_' + tab + '_' + subt
            tot_err = []
            for c in range(0,len(df_comparison.columns)):
                errbycol = df_comparison[c].str.count("<>").sum()
                tot_err.append(errbycol)
            perc_err = sum(tot_err)/(len(df_comparison.index) * len(df_comparison.columns))
            recap_sheet['C' + str(r+5)].value = round(perc_err,2)
        except:
            pass

    def err_handler(r):
        
        sheet_err = workbook.create_sheet(str(r))
        sheet_err.sheet_view.showGridLines = False
        sheet_err['B2'].value = 'Unable to compare : reports have different number of rows/columns!'
        sheet_err['B3'].value = 'Check out the raw reports in the output folder in ' + path0
        recap_sheet['B' + str(r+5)].value = str(r) + '_' + check + '_' + ptf + '_' + tab + '_' + subt
        recap_sheet['C' + str(r+5)].value = 'Error: reports with different # of rows/columns'
    
    def err_handler_manual(r):
        
        pag.FAILSAFE = False
        sheet_err = workbook.create_sheet(str(r))
        sheet_err.sheet_view.showGridLines = False
        sheet_err['B2'].value = 'Interrupted manually by moving the mouse to the corner of the screen'
        recap_sheet['B' + str(r+5)].value = str(r) + '_' + check + '_' + ptf + '_' + tab + '_' + subt
        recap_sheet['C' + str(r+5)].value = 'Interruppted manually'
        workbook.save(final_file)
        pag.FAILSAFE = True
    
    def iteration_err_handler(r):
        
        pag.FAILSAFE = False
        bbg_rt.bbg_window_select()
        time.sleep(1)
        
        myScreenshot = pag.screenshot()
        myScreenshot.save(path_fails + '\\' + str(r) + '_' + check + '_' + ptf + '_' + tab + '_' + subt + '.png')
        
        bbg_rt.bbg_window_select()
        time.sleep(1)
        
        pag.press('esc')
        pag.press('esc')
        pag.press('esc')
        
        try:
            time.sleep(1)
            w.find_window_wildcard(".*grid.*")
            w.set_foreground()
            bbg_rt.closeExcel()
        except:
            pass


        sheet_err = workbook.create_sheet(str(r))
        sheet_err.sheet_view.showGridLines = False
        sheet_err['B2'].value = 'Something went wrong, please check screenshot ' + str(r) +' in FAILURES folder'
        recap_sheet['B' + str(r+5)].value = str(r) + '_' + check + '_' + ptf + '_' + tab + '_' + subt
        recap_sheet['C' + str(r+5)].value = 'Something went wrong'
        pag.FAILSAFE = True

        
'''###################### Create new types of tests #####################'''


class test_types:
                
                
    def PROD_vs_QA(r):
        
        
        bbg_rt.selectptf()
        bbg_rt.openviewprod()
        bbg_rt.chooseTab()
        bbg_rt.change_bmk(2)
        bbg_rt.change_bkdn(3)
        bbg_rt.change_ccy(4)
        bbg_rt.change_day(5)
        bbg_rt.change_model(8)
        bbg_rt.change_unit(9)
        bbg_rt.change_scen(9)
        bbg_rt.change_hz(10)
        bbg_rt.change_attrb(12)

        bbg_rt.export()
        bbg_rt.export_loop()
        bbg_rt.dumpXLS1(r)
        bbg_rt.closeGridfile()

        bbg_rt.openviewqa()
        bbg_rt.chooseTab()
        bbg_rt.change_bmk(2)
        bbg_rt.change_bkdn(3)
        bbg_rt.change_ccy(4)
        bbg_rt.change_day(5)
        bbg_rt.change_model(9)
        bbg_rt.change_unit(10)
        bbg_rt.change_scen(10)
        bbg_rt.change_hz(11)
        bbg_rt.change_attrb(12)

        bbg_rt.export()
        bbg_rt.export_loop()
        bbg_rt.dumpXLS2(r)
        bbg_rt.closeGridfile()


        analyze.compute_differences(r)
        analyze.dump_results_to_excel(r)
        analyze.add_formatting()
        analyze.recap_results(r)
                       

    def V7_vs_V8(r):
        
        bbg_rt.selectptf()
        bbg_rt.openviewqa()
        bbg_rt.chooseTab()
        bbg_rt.change_bmk(2)
        bbg_rt.change_bkdn(3)
        bbg_rt.change_ccy(4)
        bbg_rt.change_day(5)
        bbg_rt.change_model(9)
        bbg_rt.change_unit(10)
        bbg_rt.change_hz(11)

        bbg_rt.export()
        bbg_rt.export_loop()
        bbg_rt.dumpXLS1(r)
        bbg_rt.closeGridfile()

        bbg_rt.change_riskversion(8)
        
        bbg_rt.change_model(9)

        bbg_rt.export()
        bbg_rt.export_loop()
        bbg_rt.dumpXLS2(r)
        bbg_rt.closeGridfile()


        analyze.compute_differences_v8(r)
        analyze.dump_results_to_excel(r)
        analyze.add_formatting()
        analyze.recap_results(r)

        
'''###################### Button Actions #####################'''
        
    
class button_actions():
    
    
    def click_to_start(b):
        
        #decalare all inputs
        global df_custom, final_file, r, w, check, tmp, ptf, bmk, tab, subt, view, day, ccy, bkdn, model, unit, clvl, scen, attrb, hz, m1, m2
        
        #close excel in case it's open
        bbg_rt.closeExcel()
        
        #set window handler
        w = WindowMgr()
        
        #open bbg window for the first time, we do this just once at the beginning cause it might cause errors later
        bbg_rt.bbg_window_open()
        
        if tests_type.value in ['CUSTOM','NXQC','RISK','HIPPO']:
            
            first_step.create_folders()
            first_step.read_inputs_from_ctrl_file()
            first_step.setup_excel_template()
            final_file = path_results + '\\REPORT_' + tests_type.value + '_' + str(datetime.now().strftime("%H%M")) + '.xlsx'
        

            for r in range(0,len(df_custom.index)):
                    check = df_custom.iloc[r,0]
                    tmp = df_custom.iloc[r,1]
                    ptf = df_custom.iloc[r,3]
                    bmk = df_custom.iloc[r,4]
                    tab = df_custom.iloc[r,5]
                    subt = df_custom.iloc[r,6]
                    view = df_custom.iloc[r,7]
                    day = df_custom.iloc[r,8]
                    ccy = df_custom.iloc[r,10]
                    bkdn = df_custom.iloc[r,11]
                    model = df_custom.iloc[r,12]
                    unit = df_custom.iloc[r,13]
                    clvl = df_custom.iloc[r,14]
                    hz = df_custom.iloc[r,15]
                    scen = df_custom.iloc[r,18]
                    m1 = str(df_custom.iloc[r,19])
                    m2 = str(df_custom.iloc[r,20])
                    
                    

                    if check == 'PROD_vs_QA':
                        
                        try:
                            
                            test_types.PROD_vs_QA(r)
                                
                        except Exception as e:
                            
                            if str(e) == 'Can only compare identically-labeled DataFrame objects':
                                analyze.err_handler(r)
                                
                            elif str(e).split(' ')[0] == 'PyAutoGUI':
                                analyze.err_handler_manual(r)
                                break

                            else:
                                analyze.iteration_err_handler(r)

                                
                        
                        
                        #save workboook after each iteration
                        workbook.save(final_file)
                        
                        

                    elif check == 'V7_vs_V8':
                        
                        try:
                            test_types.V7_vs_V8(r)
                            
                        except Exception as e:
                            
                            if str(e) == 'Can only compare identically-labeled DataFrame objects':
                                analyze.err_handler(r)
                                
                            elif str(e).split(' ')[0] == 'PyAutoGUI':
                                analyze.err_handler_manual(r)
                                break

                            else:
                                analyze.iteration_err_handler(r)
                                
                        #save workboook after each iteration
                        workbook.save(final_file)
                    

            
            time.sleep(5)
            #turnoff computer after long run if flagged, otherwise open results
            if turnoff_button.value == True:
                os.system("shutdown /s /t 1")
            else:
                master = Tk()
                master.withdraw()  
                messagebox.showinfo("BBG_RT","Process Completed")
                os.startfile(final_file)

        else:
            
            first_step.create_folders()
            first_step.setup_excel_template()
            
            #read inputs from UI
            check = tests_type.value
            ptf = ptfs_list.value
            tab = tabs_list.value
            view = views_list.value
            subt = subtabs_list.value
            bmk = bmks_list.value
            day = days_list.value
            ccy = ccys_list.value
            bkdn = bkdns_list.value
            model = models_list.value
            unit = units_list.value
            clvl = clvls_list.value
            hz = hzs_list.value
            m2 = qa_machines.value
            tmp = tmps_list.value
            scen = scens_list.value

       
            if tests_type.value == 'PROD_vs_QA':
            
                try:
                    test_types.PROD_vs_QA(r = 0)
                    
                except Exception as e:
                            
                            if str(e) == 'Can only compare identically-labeled DataFrame objects':
                                analyze.err_handler(r)
                                
                            elif str(e).split(' ')[0] == 'PyAutoGUI':
                                analyze.err_handler_manual(r)

                            else:
                                analyze.iteration_err_handler(r)

            elif tests_type.value == 'V7_vs_V8':
                
                try:
                    test_types.V7_vs_V8(r = 0)
                    
                except Exception as e:
                            
                            if str(e) == 'Can only compare identically-labeled DataFrame objects':
                                analyze.err_handler(r)
                                
                            elif str(e).split(' ')[0] == 'PyAutoGUI':
                                analyze.err_handler_manual(r)

                            else:
                                analyze.iteration_err_handler(r)
                                
                                
            final_file = path_results + '\\report_' + check + '_' + str(datetime.now().strftime("%H%M")) + '.xlsx' 
            workbook.save(final_file)
            master = Tk()
            master.withdraw()  
            messagebox.showinfo("BBG_RT","Process Completed")
            os.startfile(final_file)
            

    def click_to_stop(b):
        
        pag.press('esc')
        pag.press('0')
        pag.press('0')
        time.slee(3)
        pag.press('enter')
    
    
'''###################### Small UI #####################'''            
    
    
class app:           

    
    def start():
        
        '''Generates the small UI with buttons and dropdowns'''
        
        global tabs_list, ptfs_list, views_list, scens_list
        global subtabs_list, tests_type, bmks_list, bkdns_list
        global ccys_list, days_list, models_list, units_list
        global clvls_list, hzs_list, qa_machines, tmps_list, turnoff_button
        
        #lists for dropdowns widgets
        tab_list = ['CH','HD','HP','PA','TE','VR','SA']
        ptf_list = ['H160819-6','H160819-12','H160819-52','U4153597-165','...']
        view_list = ['NX_EQUITY','NX_FIXINC','NX_BALANCED']
        subtab_list = ['MainView',' ','--TE tabs--','Summary', 'Factors', 'RiskBets', 'Trends', 'Exposure',
                      ' ','--HP tabs--','TotalReturn','PeriodAnalysis','SeasonalAnalysis','StatisticalSummary']
        check_type = ['CUSTOM','NXQC','RISK','HIPPO',' ',
                      '--Or select a single custom test--','PROD_vs_QA','V7_vs_V8']
        qa_machine = ['2973','2974','2977']
        bmk_list = ['Default', 'None', 'MXWO', 'INDU']
        bkdn_list = ['Default', 'None', 'Market Cap Ranges', 'Security Type']
        ccy_list = ['Default', 'USD', 'EUR', 'GBP', 'JPY', 'CHF', 'CAD', 'HKD']
        day_list = ['PREV_CLOSE','12/31/19','01/16/20','01/31/20','...']
        model_list = ['Default', ' ','----Risk Models----', 'Asian', 'Australian', 'Canadian',
                      'Chinese', 'Emerging','European', 'Global', 'Japanese', 'Latin America',
                      'US', 'Fixed Income','Bloomberg Risk Model (Global)', 'Bloomberg Risk Model (Regional)',
                      ' ','----Attribution Models----', 'Total Ret', 'Spread Ret','Excess Ret', 'Factor-Base']
        unit_list = ['Default', 'P&L', 'Returns', 'Basis Points']
        clvl_list = ['Default', '90', '95', '99']
        hz_list = ['Default', '1 Day', '1 Week', '2 Weeks', '1 Month', '1 Quarter','1 Year']
        tmp_list = ['Current Tab (Unformatted xls)', 'Current Tab (xls)']
        scen_list = ['Default','All Scenarios', 'Equity Markets', 'Greece', 'Libya', 'Russian', 'Japan','Lehman']
        
        #Buttons/Dropdowns
        tests_type = widgets.Dropdown(
            options=check_type,
            description='Test type : ',
            disabled=False)
        
        tabs_list = widgets.Dropdown(
            options=tab_list,
            description='Function : ',
            disabled=False)
        
        subtabs_list = widgets.Dropdown(
            options=subtab_list,
            description='Tab : ',
            disabled=False)
        
        ptfs_list = widgets.Dropdown(
            options=ptf_list,
            description='PTF #: ',
            disabled=False)
        
        views_list = widgets.Dropdown(
            options = view_list,
            description='Views : ',
            disabled=False)   

        qa_machines = widgets.Dropdown(
            options=qa_machine,
            description='QA mach : ',
            disabled=False)
        
        bmks_list = widgets.Dropdown(
            options=bmk_list,
            description='Benchmark : ',
            disabled=False,
            value = 'Default')
        
        bkdns_list = widgets.Dropdown(
            options=bkdn_list,
            description='Breakdown : ',
            disabled=False,
            value = 'Default')
        
        ccys_list = widgets.Dropdown(
            options=ccy_list,
            description='Currency : ',
            disabled=False,
            value = 'Default')
        
        days_list = widgets.Dropdown(
            options=day_list,
            description='Day : ',
            disabled=False,
            value = 'PREV_CLOSE')
        
        models_list = widgets.Dropdown(
            options=model_list,
            description='Model : ',
            disabled=False,
            value = 'Default')
        
        units_list = widgets.Dropdown(
            options=unit_list,
            description='Unit : ',
            disabled=False,
            value = 'Default')
        
        clvls_list = widgets.Dropdown(
            options=clvl_list,
            description='Conf. Level : ',
            disabled=False,
            value = 'Default')
        
        hzs_list = widgets.Dropdown(
            options=hz_list,
            description='Horizon : ',
            disabled=False,
            value = 'Default')
        
        tmps_list = widgets.Dropdown(
            options=tmp_list,
            description='Temp : ',
            disabled=False)
        
        scens_list = widgets.Dropdown(
            options=scen_list,
            description='Scenario : ',
            disabled=False)
        
        start_button = widgets.Button(
            description='START TEST',
            disabled=False,
            button_style='danger',
            layout=widgets.Layout(width='70%'))
        
        stop_button = widgets.Button(
            description='STOP TEST',
            disabled=False,
            button_style='info',
            layout=widgets.Layout(width='70%'))
        
        turnoff_button = widgets.Checkbox(
            value=False,
            description='Turn off pc after long run')
        
        hidden_button = widgets.Button(
            description='hidden',
            layout=widgets.Layout(width='50%'))
        
        hidden_button.layout.visibility = 'hidden'

        start_button.on_click(button_actions.click_to_start)
        stop_button.on_click(button_actions.click_to_stop)
        
        #Boxes containing buttons
        hidden_box = widgets.HBox([hidden_button])
        box0 = widgets.HBox([tests_type, turnoff_button])
        box1 = widgets.HBox([ptfs_list, tabs_list, views_list])
        box2 = widgets.HBox([subtabs_list, bmks_list, bkdns_list])
        box3 = widgets.HBox([ccys_list, days_list, units_list])
        box4 = widgets.HBox([models_list, clvls_list, hzs_list ])
        box5 = widgets.HBox([scens_list, qa_machines, tmps_list])
        box_start = widgets.HBox([hidden_button,start_button,hidden_button])
        box_stop = widgets.HBox([hidden_button,stop_button,hidden_button])
        
        
        finalUI = widgets.VBox([box0,hidden_button,box1,box2,box3,box4,box5,hidden_box,box_start])
            
        return finalUI  
    


In [6]:
# shuts off useless warnings
warnings.filterwarnings("ignore", message = "Title is more than 31 characters. Some applications may not be able to read the file")

#print titles and legenda
printred('-----------------------------------------')
printred('\033[1m'+' PORT UI REGRESSION TESTER by MMAIOCCHI1'+'\033[1m')
printred('-----------------------------------------')
print(' ')

#launch UI
app.start()

 -----------------------------------------
  PORT UI REGRESSION TESTER by MMAIOCCHI1
 -----------------------------------------
 
